# Paths

In [1]:
path = '/home/tih_isi_7/G-NeuroDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'
path_tab = path + 'Results/Tables/'

# Importing libraries

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y['x'], n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn.to_csv(path_tab+dataname+'_'+methodname+'_knn.csv')
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.791667,0.785225,0.791667,0.782398
neighbour 15,0.763889,0.757420,0.763889,0.746321
neighbour 25,0.743056,0.747535,0.743056,0.725264
neighbour 35,0.736111,0.741925,0.736111,0.719041
neighbour 45,0.718750,0.713187,0.718750,0.698008


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y['x'], n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in n_tree],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc.to_csv(path_tab+dataname+'_'+methodname+'_rfc.csv')
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 20,0.763889,0.758345,0.763889,0.753899
n_tree 40,0.767361,0.766890,0.767361,0.761949
n_tree 60,0.763889,0.759907,0.763889,0.756353
n_tree 80,0.774306,0.771698,0.774306,0.766477
n_tree 100,0.777778,0.776129,0.777778,0.769756


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y['x']))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:01<00:00,  8.91it/s]


,ARI,FMI,NMI
0,0.529609,0.711643,0.554321
1,0.515346,0.703392,0.540568
2,0.495042,0.696031,0.521984
3,0.506761,0.703102,0.532979
4,0.509824,0.704343,0.535864
5,0.525488,0.717239,0.549921
6,0.510304,0.705365,0.536739
7,0.506047,0.702941,0.532317
8,0.512786,0.704636,0.538398
9,0.520216,0.707365,0.545191


### Agglomerative 

In [7]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y['x']), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T

,ARI,FMI,NMI
0,0.535991,0.725557,0.561619
